# 1. Query tools

This Section introduces three methods to query the ALMA archive:

- [1.1 - Query by target name (<i>alminer.target</i>)](#1.1-Query-by-target-name)<br>
- [1.2 - Query a catalog (<i>alminer.catalog</i>)](#1.2-Query-by-position)<br>
- [1.3 - Query by ALMA keywords (<i>alminer.keysearch</i>)](#1.3-Query-by-ALMA-keywords)

<h3>General notes about the querying functions:</h3>

* All querying functions search the ALMA archive for public data by default. To include both public and proprietary data in the search, set *public=None*. Similarly, to search for only propietary data, set *public=False*. 
* All querying functions search the ALMA archive for both published and unpublished data. To include only unpublished data, set *published=False*.
* The querying functions will by default print a summary of the observations, including a list of target names. For large queries, it is useful to turn this feature off in order to not have a long list of targets printed to screen. To turn off this feature, simply set *print_targets=False*.
* The queries return all possible observations in [PANDAS DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html) format that can be used to further narrow down your search as demonstrated in [Section 2](2_filter_explore.ipynb).

<h3>Load libraries </h3>

In [1]:
import alminer
import pandas
from astropy.io import ascii

## 1.1 Query by target name

The [<code>alminer.target</code>](../pages/api.rst#alminer.target) function allows one to query objects by name. This function uses the Astropy [SESAME resolver](http://cds.u-strasbg.fr/cgi-bin/Sesame) which searches multiple databases (Simbad, NED, VizieR) to obtain the coordinates of the object of interest, and then queries the ALMA archive for those coordinates and a search radius around them. The search radius is by default 1.0 arcminute, but can be modified using the *search_radius* keyword (in arcmin units). 

### Example 1.1.1: query two sources by name

In [2]:
myquery = alminer.target(['Orion KL', "AB Aur"])

alminer.target results 
Target = Orion KL
--------------------------------
Number of projects = 27
Number of observations = 63
Number of unique subbands = 226
Total number of subbands = 272
23 target(s) with ALMA data = ['Orion H2O maser outburst', 'OrionKL', 'Orion_Source_I', 'Orion KL', 'Orion-KL', 'Orion_KL', 'BN-KL', 'OrionKL-SV', 'f15', 'f14', 'OMC1_NW', 'Orion1', 'OMC1_SE', 'BN', 'f23', 'Orion', 'Orion_BNKL_source_I', 'OMC-1_Region2', 'orion_kl', 'orion-IRc2', 'Orion_KL_Field_1_Orion_Hot_Core', 'Orion_KL_Field_3_North-west_Clump', 'Orion_KL_Field_2_SMA1']
--------------------------------
Target = AB Aur
--------------------------------
Number of projects = 3
Number of observations = 3
Number of unique subbands = 17
Total number of subbands = 17
3 target(s) with ALMA data = ['ab_aurigae', 'AB_Aur', 'AB_Auriga']
--------------------------------


Now let's increase the search radius:

In [3]:
myquery = alminer.target(['Orion KL', "AB Aur"], search_radius=2.0)

alminer.target results 
Target = Orion KL
--------------------------------
Number of projects = 35
Number of observations = 100
Number of unique subbands = 283
Total number of subbands = 451
46 target(s) with ALMA data = ['HC602_HC606_HC608', 'Orion H2O maser outburst', 'OrionField1-2', 'OrionKL', 'Orion_Source_I', 'HC672', 'OrionField1-1', 'OrionField2', 'Orion KL', 'OMC-1S', 'Orion-KL', '104', 'Orion_KL', '71', 'BN-KL', 'OrionKL-SV', 'f11', 'f12', 'f15', 'f13', 'f14', 'OMC1_SE', 'OMC1_NW', 'f10', 'Orion1', 'f8', 'f4', 'f5', 'f1', 'f7', 'f3', 'BN', 'f23', 'Orion', 'Orion_BNKL_source_I', 'f16', 'OMC-1_Region4', 'OMC-1_Region2', 'orion_kl', 'ONC_Mosaic', 'orion-IRc2', 'Orion_KL_Field_1_Orion_Hot_Core', 'ONC', 'Orion_KL_Field_3_North-west_Clump', 'Orion_KL_Field_2_SMA1', 'ORS-8']
--------------------------------
Target = AB Aur
--------------------------------
Number of projects = 3
Number of observations = 3
Number of unique subbands = 17
Total number of subbands = 17
3 target(s) with A

### Example 1.1.2: query a list of objects by name

First create a catalog or a list of object names. In this example, the catalog `Sample_cat.dat` has the following content:
```
 Name     RA      DEC   
------ -------- --------
AB_Aur  73.9412  30.5511
AK_Sco 253.6867 -36.8886
AS_310 278.3383  -4.9683
AS_470 324.0592  57.3586
AS_477 328.1421  47.2289
```

Note that the column that is used is the *Name* column and the coordinates are ignored in this example.

In [4]:
mylist = ascii.read("Sample_cat.dat", header_start=0, data_start=1)
myquery = alminer.target(mylist['Name'])

alminer.target results 
Target = AB_Aur
--------------------------------
Number of projects = 3
Number of observations = 3
Number of unique subbands = 17
Total number of subbands = 17
3 target(s) with ALMA data = ['ab_aurigae', 'AB_Aur', 'AB_Auriga']
--------------------------------
Target = AK_Sco
--------------------------------
Number of projects = 3
Number of observations = 3
Number of unique subbands = 12
Total number of subbands = 12
2 target(s) with ALMA data = ['AK_Sco', 'HIP_82747']
--------------------------------
Target = AS_310
--------------------------------
No observations found.
--------------------------------
Target = AS_470
--------------------------------
No observations found.
--------------------------------
Target = AS_477
--------------------------------
No observations found.
--------------------------------


### Example 1.1.3: include proprietary data

In [5]:
myquery = alminer.target(mylist['Name'], public=None)

alminer.target results 
Target = AB_Aur
--------------------------------
Number of projects = 4
Number of observations = 5
Number of unique subbands = 25
Total number of subbands = 25
3 target(s) with ALMA data = ['ab_aurigae', 'AB_Aur', 'AB_Auriga']
--------------------------------
Target = AK_Sco
--------------------------------
Number of projects = 4
Number of observations = 4
Number of unique subbands = 16
Total number of subbands = 16
2 target(s) with ALMA data = ['AK_Sco', 'HIP_82747']
--------------------------------
Target = AS_310
--------------------------------
No observations found.
--------------------------------
Target = AS_470
--------------------------------
No observations found.
--------------------------------
Target = AS_477
--------------------------------
No observations found.
--------------------------------


### Example 1.1.4: account for mosaics

The [<code>alminer.target</code>](../pages/api.rst#alminer.target) function will by default search whether the phase center of any ALMA observations lie within the search radius around the target of interest's position. To search whether any part of the observed region overlaps with the cone extending the search radius, one can set *point=False*. Note that this option is slower, hence it's not the dafault mode.

In [6]:
myquery = alminer.target(['Orion KL'], point=False)

alminer.target results 
Target = Orion KL
--------------------------------
Number of projects = 35
Number of observations = 90
Number of unique subbands = 283
Total number of subbands = 399
40 target(s) with ALMA data = ['HC602_HC606_HC608', 'Orion H2O maser outburst', 'OrionField1-2', 'OrionKL', 'Orion_Source_I', 'OrionField1-1', 'Orion KL', 'Orion-KL', '104', 'Orion_KL', 'BN-KL', 'OrionKL-SV', 'f11', 'f12', 'f15', 'f13', 'f14', 'OMC1_SE', 'OMC1_NW', 'f10', 'Orion1', 'f1', 'f3', 'BN', 'f23', 'Orion', 'OMC-1_Region5', 'Orion_BNKL_source_I', 'f16', 'OMC-1_Region4', 'OMC-1_Region2', 'orion_kl', 'OMC-1', 'ONC_Mosaic', 'orion-IRc2', 'Orion_KL_Field_1_Orion_Hot_Core', 'ONC', 'Orion_KL_Field_3_North-west_Clump', 'Orion_KL_Field_2_SMA1', 'ORS-8']
--------------------------------


## 1.2 Query by position

The [<code>alminer.conesearch</code>](../pages/api.rst#alminer.conesearch) and [<code>alminer.catalog</code>](../pages/api.rst#alminer.catalog) functions can be used to directly query the ALMA archive by positions in the sky and a search radius around them. The right ascension and declinations must be given in units of degrees (ICRS). You can use the [Astropy coordinates package](https://docs.astropy.org/en/stable/coordinates/index.html) to convert your desired coordinates to degrees.

### Example 1.2.1:  query an object by its coordinates (RA, Dec)

In [7]:
myquery = alminer.conesearch(ra=201.365063, dec=-43.019112, search_radius=10.0)

--------------------------------
Number of projects = 23
Number of observations = 75
Number of unique subbands = 190
Total number of subbands = 305
9 target(s) with ALMA data = ['CenA', 'J1325-430', 'Centaurus_a', 'J1325-4301', 'Centaurus A', 'Centaurus_A', '3FGL_J1325.4-4301', 'NGC_5128', 'Cen_A']
--------------------------------


### Example 1.2.2:  query a catalog of objects by their coordinates (RA, Dec)

Let's first import a catalog, for example the catalog of Spitzer YSOs in Orion from Megeath et al. (2009), and create a PANDAS DataFrame using rows 866 to 869 of this catalog. Then use the [<code>alminer.catalog</code>](../pages/api.rst#alminer.catalog) function to query the ALMA archive for each target in the DataFrame.

In [8]:
Spitzer = ascii.read("Spitzer_sample.dat", header_start=0, data_start=866, data_end=869)

mycat =  {"Name": Spitzer['Seq'], 
          "RAJ2000" : Spitzer["RA2000"], 
          "DEJ2000" : Spitzer["DEC2000"]}

mycat = pandas.DataFrame(mycat)

myquery = alminer.catalog(mycat, search_radius=1.)

alminer.catalog results
Target = 866
--------------------------------
Number of projects = 1
Number of observations = 1
Number of unique subbands = 4
Total number of subbands = 4
1 target(s) with ALMA data = ['M12_866']
--------------------------------
Target = 867
--------------------------------
Number of projects = 1
Number of observations = 1
Number of unique subbands = 4
Total number of subbands = 4
1 target(s) with ALMA data = ['M12_867']
--------------------------------
Target = 868
--------------------------------
Number of projects = 2
Number of observations = 2
Number of unique subbands = 8
Total number of subbands = 8
1 target(s) with ALMA data = ['HOPS-172']
--------------------------------


## 1.3 Query by ALMA keywords

Query the ALMA archive for any [(string-type) keywords defined in ALMA TAP system](../pages/query_keywords.rst) using the [<code>alminer.keysearch</code>](../pages/api.rst#alminer.keysearch) function.

The power of this function is in combining keywords. When multiple keywords are provided, they are queried using 'AND' logic, while when multiple values are provided for a given keyword, they are queried using 'OR' logic. For example,

 * `alminer.keysearch({"proposal_abstract": ["high-mass star formation outflow disk"]})`
will query the archive for projects with the words "high-mass" AND "star" AND "formation" AND "outflow" AND "disk" in their proposal abstracts.
<br>

 * `alminer.keysearch({"proposal_abstract": ["high-mass", "star", "formation", "outflow", "disk"]})`
will query the archive for projects with the words "high-mass" OR "star" OR "formation" OR "outflow" OR "disk" in their proposal abstracts.
<br>

 * `alminer.keysearch({"proposal_abstract": ["star formation"], "scientific_category":['Galaxy evolution']})`
will query the archive for projects with the words "star" AND "formation" in their proposal abstracts AND projects that are within the scientific_category of 'Galaxy evolution'.


<div class="alert alert-info">

Note

* Tables of ALMA [scientific categories](../pages/scientific_categories.rst) and [science keywords](../pages/science_keywords.rst) are provided on the sidebar. 

* For an overview, see [Appendix D of the ALMA Proposer’s
Guide](https://almascience.eso.org/proposing/proposers-guide#section-63).

</div>

### Example 1.3.1: query a list of ALMA target names that may not be in SIMBAD/NED/VizieR

In [9]:
myquery = alminer.keysearch({'target_name': ['GRB021004','SPT0319-47', 'G345']})

alminer.keysearch results 
--------------------------------
Number of projects = 16
Number of observations = 28
Number of unique subbands = 121
Total number of subbands = 179
10 target(s) with ALMA data = ['GRB021004', 'G345.5', 'SPT0319-47', 'G345.5043+00.3480', 'G345.49+1.47', 'G345.50+0.35', 'G345.6487+0.0089', 'G345.01', 'G345.11', 'G345.0029-0.2241']
--------------------------------


### Example 1.3.2: query a list of ALMA projects by their proposal IDs

In [10]:
myquery = alminer.keysearch({'proposal_id': ['2015.1.00664.S', '2016.1.00204.S']})

alminer.keysearch results 
--------------------------------
Number of projects = 2
Number of observations = 16
Number of unique subbands = 16
Total number of subbands = 64
16 target(s) with ALMA data = ['KMOS3DGS4-27882', 'KMOS3DCOS4-24763', 'KMOS3DU4-34138', 'KMOS3DCOS4-19680', 'KMOS3DCOS4-13701', 'KMOS3DCOS4-10347', 'KMOS3DCOS4-13174', 'KMOS3DCOS4-15820', 'KMOS3DCOS4-15813', 'KMOS3DU4-22227', 'KMOS3DU4-20547', 'KMOS3DU4-32147', 'KMOS3DGS4-24110', 'KMOS3DGS4-11016', 'KMOS3DGS4-25151', 'AK_Sco']
--------------------------------


### Example 1.3.3: query by words in the proposal abstract

Query the ALMA archive for proposals that have the words 'high-mass' AND 'star' AND 'formation' AND 'outflow', and do not print the long list of target names in the summary.

In [11]:
myquery = alminer.keysearch({'proposal_abstract': ['high-mass star formation outflow']}, 
                            print_targets=False)

alminer.keysearch results 
--------------------------------
Number of projects = 34
Number of observations = 419
Number of unique subbands = 416
Total number of subbands = 2799
Total number of targets with ALMA data = 239
--------------------------------


### Example 1.3.4: query by combination of keywords

Query the ALMA archive for proposals that have the words ('high-mass' AND 'star' AND 'formation' AND 'outflow') OR ('massive' AND 'star' AND 'formation') corresponding to the scientific category of 'Galaxy evolution'.

In [12]:
myquery = alminer.keysearch({'proposal_abstract': ['high-mass star formation', 'massive star formation'], 
                             'scientific_category':['Galaxy evolution']}, print_targets=False)

alminer.keysearch results 
--------------------------------
Number of projects = 119
Number of observations = 2056
Number of unique subbands = 2481
Total number of subbands = 8247
Total number of targets with ALMA data = 1376
--------------------------------


### Example 1.3.5: query for full polarization data

In [13]:
myquery = alminer.keysearch({'science_keyword':['disks around low-mass stars'], 
                             'pol_states':['XY', 'YX']}, print_targets=False)

alminer.keysearch results 
--------------------------------
Number of projects = 27
Number of observations = 48
Number of unique subbands = 94
Total number of subbands = 192
Total number of targets with ALMA data = 34
--------------------------------
